In [3]:
from sklearn.preprocessing import StandardScaler 
import pandas as pd
import numpy as np
import tensorflow as tf
tf.__version__

'2.4.0'

In [4]:
dataset_sz = 5000
csv = "synth_nonlinear.csv"
df = pd.read_csv(csv)
df_test = df.iloc[-1000:]
df = df.iloc[:dataset_sz]

scaler = StandardScaler()
n_folds = 10
df = scaler.fit_transform(df)
df_test = scaler.transform(df_test)

X_test = df_test
y_test = df_test[:,0]
X_DAG = df
y_DAG = np.expand_dims(X_DAG[:,0], -1)

In [5]:
## Hyperparameters
num_train = X_DAG.shape[0]
lr  = None
batch_size = 32
num_inputs = X_DAG.shape[1]
num_outputs = 1
w_threshold = 0.3
n_hidden = 10
# hidden_layers = 2
ckpt_file = 'tmp.ckpt'
# standardize = True
reg_lambda=1
reg_beta=5
# DAG_min = 0.5
learning_rate = 0.001

max_steps = 200
saves = 50 
patience = 30
# metric = mean_squared_error
seed = 1


In [7]:
## Create input tensors
X = tf.Variable(initial_value = X_DAG, shape=[None, num_inputs], dtype=tf.dtypes.float32)
y = tf.Variable(initial_value = y_DAG, dtype=tf.dtypes.float32, shape=[None, 1])
X, y

# rho =  tf.placeholder("float",[1,1])
# alpha =  tf.placeholder("float",[1,1])
# keep_prob = tf.placeholder("float")
# Lambda = tf.placeholder("float")
# noise = tf.placeholder("float")
# is_train = tf.placeholder(tf.bool, name="is_train")

# One-hot vector indicating which nodes are trained
# sample = tf.Variable(dtype=tf.int32, shape=[num_inputs])
X, y

(<tf.Variable 'Variable:0' shape=(None, 10) dtype=float32, numpy=
 array([[-1.484536  , -1.5275788 ,  0.48957345, ..., -0.9901357 ,
         -0.68122566,  0.09370673],
        [ 2.0711331 , -0.77660966,  0.5837853 , ...,  1.7845517 ,
          0.45600224, -0.7173038 ],
        [-0.36621284, -0.68882316,  0.8255042 , ...,  0.97385097,
          1.1061857 , -1.1758687 ],
        ...,
        [-0.70872915, -1.0960492 ,  0.5624519 , ..., -1.8072734 ,
          0.40068227, -1.2580615 ],
        [-0.81493574, -0.45984966, -0.15990469, ..., -0.45329618,
         -0.30793667, -0.6496865 ],
        [ 1.2409345 ,  0.7310152 , -0.8439071 , ...,  1.3917621 ,
          0.09549129,  0.75522476]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(None, 1) dtype=float32, numpy=
 array([[-1.484536  ],
        [ 2.0711331 ],
        [-0.36621284],
        ...,
        [-0.70872915],
        [-0.81493574],
        [ 1.2409345 ]], dtype=float32)>)

In [8]:
# Store layers weight & bias
weights = {}
biases = {}

# Create the input and output weight matrix for each feature
for i in range(num_inputs):
    weights['w_h0_'+str(i)] = tf.Variable(tf.random.normal([num_inputs, n_hidden], seed = seed))
    weights['out_'+str(i)] = tf.Variable(tf.random.normal([n_hidden, num_outputs], seed = seed))

for i in range(num_inputs):
    biases['b_h0_'+str(i)] = tf.Variable(tf.random.normal([n_hidden], seed = seed))
    biases['out_'+str(i)] = tf.Variable(tf.random.normal([num_outputs], seed = seed))

# The first layer is shared
weights.update({
    'w_h1': tf.Variable(tf.random.normal([n_hidden, n_hidden]))
})

biases.update({
    'b_h1': tf.Variable(tf.random.normal([n_hidden]))
})

# weights
# biases

In [9]:
## Create two hidden layer network with masking
## one output per input.. reconstructing the inputs

hidden_h0 = {}
hidden_h1 = {}
layer_1 = {}
layer_1_dropout = {}
out_layer = {}

Out_0 = []

# Mask removes the feature i from the network that is tasked to construct feature i
mask = {}
activation = tf.nn.relu

for i in range(num_inputs):
    indices = [i]*n_hidden
    mask[str(i)] = tf.transpose(tf.one_hot(indices, depth=num_inputs, on_value=0.0, off_value=1.0, axis=-1))

    weights['w_h0_'+str(i)] = weights['w_h0_'+str(i)]*mask[str(i)] 
    hidden_h0['nn_'+str(i)] = activation(tf.add(tf.matmul(X, weights['w_h0_'+str(i)]), biases['b_h0_'+str(i)]))
    hidden_h1['nn_'+str(i)] = activation(tf.add(tf.matmul(hidden_h0['nn_'+str(i)], weights['w_h1']), biases['b_h1']))
    out_layer['nn_'+str(i)] = tf.matmul(hidden_h1['nn_'+str(i)], weights['out_'+str(i)]) + biases['out_'+str(i)]
    Out_0.append(out_layer['nn_'+str(i)])

# Concatenate all the constructed features
Out = tf.concat(Out_0,axis=1)

# weights
# hidden_h1
# out_layer
# Out_0
# Out


In [10]:
## Adjeciency matrix
## L2 norm of W
W_0 = []
for i in range(num_inputs):
    W_0.append(tf.math.sqrt(tf.reduce_sum(tf.square(weights['w_h0_'+str(i)]),axis=1,keepdims=True)))

W = tf.concat(W_0,axis=1)
W

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0.       , 2.9891484, 2.9120748, 2.607893 , 2.2550366, 3.9059763,
        2.974898 , 2.0598476, 3.0118651, 3.3969736],
       [3.258621 , 0.       , 2.1467886, 3.4455414, 3.7271032, 4.587194 ,
        3.1495423, 3.4206116, 2.7484396, 3.1217365],
       [3.8992577, 3.861907 , 0.       , 4.720157 , 3.374345 , 2.5452418,
        2.3650439, 3.796946 , 2.6909115, 4.1147914],
       [2.9020035, 1.8003858, 2.5942261, 0.       , 4.149866 , 3.6990008,
        4.134482 , 3.070836 , 2.6891289, 3.1626332],
       [2.7818089, 3.53201  , 2.3762486, 3.468664 , 0.       , 4.06795  ,
        1.8370348, 3.1209235, 3.4086907, 1.71804  ],
       [3.345568 , 2.8428469, 3.0271053, 2.1238883, 4.9395933, 0.       ,
        2.1189463, 2.6840472, 3.9657218, 3.0005105],
       [3.7834094, 3.3540597, 4.2328653, 3.0827007, 2.6465695, 3.3646953,
        0.       , 2.60578  , 3.2701423, 4.304406 ],
       [2.5742152, 2.8836102, 2.2753868, 3.1720796, 2.8149788

In [11]:
## R_W

d = tf.cast(X.shape[1], tf.float32)
coff = 1.0 
Z = tf.multiply(W,W)

dag_l = tf.cast(d, tf.float32) 

Z_in = tf.eye(d)

#truncated power series (truncated at 10)
for i in range(1,10):

    Z_in = tf.matmul(Z_in, Z)
#     print(Z_in)
#     print(tf.linalg.trace(Z_in))
    
    dag_l += 1./coff * tf.linalg.trace(Z_in)
    
#     print(coff, 1./coff)
    
    coff = coff * (i+1)
    
## R_W
h = dag_l - tf.cast(d, tf.float32)
h

<tf.Tensor: shape=(), dtype=float32, numpy=1789651000000.0>

In [13]:
## V_W
L1_loss = 0.0
for i in range(num_inputs):
    print('weigths', weights['w_h0_'+str(i)])
    w_1 = tf.slice(weights['w_h0_'+str(i)],[0,0],[i,-1])
    print('w_1',w_1,tf.reduce_sum(tf.norm(w_1,axis=1)))
    w_2 = tf.slice(weights['w_h0_'+str(i)],[i+1,0],[-1,-1])
    print('w_2',w_2,tf.reduce_sum(tf.norm(w_2,axis=1)))
    L1_loss += tf.reduce_sum(tf.norm(w_1,axis=1))+tf.reduce_sum(tf.norm(w_2,axis=1))
#     print('L1_loss',L1_loss)

weigths tf.Tensor(
[[-0.          0.          0.         -0.          0.          0.
   0.         -0.         -0.         -0.        ]
 [ 0.6435448  -0.26432407  1.8566332   0.5678417  -0.3828359  -1.4853433
   1.2617711  -0.02530608 -0.2646297   1.5328138 ]
 [-1.7429771  -0.43789294 -0.56601     0.32066926  1.132831   -2.2782571
   0.48281202 -1.3127087   0.35685033 -1.7302834 ]
 [-0.04016773  0.8996619  -1.3805891   1.4814624  -0.2454948  -0.7326472
  -0.19589645  0.07170801  0.6329809  -1.5711907 ]
 [ 1.3293812  -1.173367    0.0315446   0.47705248  0.43694198 -0.3168089
  -0.45075032 -1.8060657   0.12489964 -0.7706542 ]
 [-0.74624157 -0.28195325 -1.9588155  -0.3376107   1.0301983   1.5134017
   0.22515805 -0.28566208  0.26882544  1.746211  ]
 [ 0.92387104 -2.0590997  -0.31438306  1.2103382   0.694803   -1.06555
   0.01364011 -1.0677125  -0.18407504 -2.2056234 ]
 [ 1.8290592   1.2431902  -0.33655512 -0.04000888 -0.33585522 -0.30744898
  -0.7669297  -0.28710833 -0.29470286 -0.8099063

In [144]:
from random import sample 
import random
# Residuals
R = X - Out 

num_nodes = np.shape(X_DAG)[1]
num_inputs= X_DAG.shape[1]
one_hot_sample = [0]*num_inputs
subset_ = sample(range(num_inputs),num_nodes) 
for j in subset_:
    one_hot_sample[j] = 1

# Divide the residual into untrain and train subset
_, subset_R = tf.dynamic_partition(tf.transpose(R), partitions=one_hot_sample, num_partitions=2)
subset_R = tf.transpose(subset_R)

subset_R

<tf.Tensor: shape=(5000, 10), dtype=float32, numpy=
array([[ 30.263233  , -14.155115  ,  -8.639004  , ..., -30.443684  ,
          9.576943  ,   7.6275373 ],
       [ 23.302568  ,  -0.694162  , -66.01288   , ..., -13.165191  ,
          0.19194162,   0.6470868 ],
       [ 28.847435  , -11.004281  , -59.29043   , ...,  -2.2444994 ,
         -1.495836  ,   5.2679815 ],
       ...,
       [ 25.28162   ,  -6.9379907 , -17.31309   , ..., -13.811258  ,
          7.228603  ,   3.171753  ],
       [ 17.581148  , -15.1001835 ,  -3.6546338 , ..., -10.713383  ,
          0.29217696,   1.2802916 ],
       [ 19.721687  ,  14.499953  , -73.23136   , ...,  12.951685  ,
          2.7972045 ,   2.2777574 ]], dtype=float32)>

In [147]:


#Combine all the loss
mse_loss_subset = tf.cast(num_inputs, tf.float32)/ tf.cast(tf.reduce_sum(one_hot_sample), tf.float32)* tf.reduce_sum(tf.square(subset_R))
regularization_loss_subset =  mse_loss_subset +  reg_beta * L1_loss +  0.5 * h * h + h

#Add in supervised loss
supervised_loss = tf.reduce_mean(tf.reduce_sum(tf.square(out_layer['nn_0'] - y),axis=1),axis=0)
# regularization_loss = 0

regularization_loss_subset +=  supervised_loss

regularization_loss_subset = tf.Variable(initial_value = regularization_loss_subset, shape=(), dtype=tf.dtypes.float32)


out_layer['nn_0']

<tf.Tensor: shape=(5000, 1), dtype=float32, numpy=
array([[-31.747768],
       [-21.231436],
       [-29.213648],
       ...,
       [-25.990349],
       [-18.396084],
       [-18.480753]], dtype=float32)>

In [138]:

optimizer_subset = tf.keras.optimizers.Adam(learning_rate=learning_rate)

loss_op_dag = optimizer_subset.minimize(var_list=None, loss=regularization_loss_subset)

# loss_op_supervised = optimizer_subset.minimize(var_list=supervised_loss + regularization_loss)

ValueError: `tape` is required when a `Tensor` loss is passed.

In [117]:



from random import sample 
rho_i = np.array([[1.0]])
alpha_i = np.array([[1.0]])

count = 0
best = 1e9
best_value = 1e9
for step in range(1, max_steps):
    h_value, loss = sess.run([h, supervised_loss], feed_dict={X: X, y: y, keep_prob : 1, rho:rho_i, alpha:alpha_i, is_train : True, noise:0})
    print("Step " + str(step) + ", Loss= " + "{:.4f}".format(loss)," h_value:", h_value) 


    for step1 in range(1, (X.shape[0] // batch_size) + 1):


        idxs = random.sample(range(X.shape[0]), batch_size)
        batch_x = X[idxs]
        batch_y = np.expand_dims(batch_x[:,0], -1)
        one_hot_sample = [0]*num_inputs
        subset_ = sample(range(num_inputs),num_nodes) 
        for j in subset_:
            one_hot_sample[j] = 1
        sess.run(loss_op_dag, feed_dict={X: batch_x, y: batch_y, sample:one_hot_sample,
                                                      keep_prob : 1, rho:rho_i, alpha:alpha_i, Lambda : reg_lambda, is_train : True, noise : 0})

    val_loss = val_loss(X_val, y_val)
    if val_loss < best_value:
        best_value = val_loss
    h_value, loss = sess.run([h, supervised_loss], feed_dict={X: X, y: y, keep_prob : 1, rho:rho_i, alpha:alpha_i, is_train : True, noise:0})
    if step >= saves:
        try:
            if val_loss < best:
                best = val_loss 
                saver.save(sess, tmp)
                print("Saving model")
                count = 0
            else:
                count += 1
        except:
            print("Error caught in calculation")

    if count > patience:
        print("Early stopping")
        break

saver.restore(sess, tmp)
W_est = sess.run(W, feed_dict={X: X, y: y, keep_prob : 1, rho:rho_i, alpha:alpha_i, is_train : True, noise:0})
W_est[np.abs(W_est) < w_threshold] = 0


def val_loss(self, X, y):
    if len(y.shape) < 2:
        y = np.expand_dims(y, -1)
    from random import sample 
    one_hot_sample = [0]*num_inputs

    # use all values for validation
    subset_ = sample(range(num_inputs),num_inputs) 
    for j in subset_:
        one_hot_sample[j] = 1

    return sess.run(supervised_loss, feed_dict={X: X, y: y, sample:one_hot_sample, keep_prob : 1, rho:np.array([[1.0]]), 
                                                          alpha:np.array([[0.0]]), Lambda : reg_lambda, is_train : False, noise:0})


def pred(self, X):
    return sess.run(out_layer['nn_0'], feed_dict={X: X, keep_prob:1, is_train : False, noise:0})

def get_weights(self, X, y):
    return sess.run(W, feed_dict={X: X, y: y, keep_prob : 1, rho:np.array([[1.0]]), alpha:np.array([[0.0]]), is_train : False, noise:0})

def pred_W(self, X, y):
    W_est = sess.run(W, feed_dict={X: X, y: y, keep_prob : 1, rho:np.array([[1.0]]), alpha:np.array([[0.0]]), is_train : False, noise:0})
    return np.round_(W_est,decimals=3)


AttributeError: module 'tensorflow._api.v2.train' has no attribute 'AdamOptimizer'